# Add MP information

In this notebook we enrich the interactions with the corresponding politician information, like name, country and sex using the hansard api.

In [1]:
import os
import requests
import json

In [ ]:
interactions = []
for file in os.listdir('interactions'):
    if file.endswith('.json'):
        with open(os.path.join('interactions', file), 'r') as f:
            interaction = json.load(f)
        
        interactions.extend(interaction)

print(len(os.listdir('interactions')), "files merged with a total of", len(interactions), "interactions")

6001 files merged with a total of 88958 interactions


In [3]:
unique_member_ids = set()
for interaction in interactions:
    unique_member_ids.add(interaction['speaker1_member_id'])
    unique_member_ids.add(interaction['speaker2_member_id'])
len(unique_member_ids)

665

Querying MP information for every single member

In [4]:
url = "https://hansard-api.parliament.uk/search/members.json"

member_info_dict = {}

for memberid in unique_member_ids:
    params = {
        "queryParameters.memberId": memberid
    }

    response = requests.get(url, params=params)

    response.raise_for_status()
    data = response.json()
    member_info = {}
    member_info['DisplayAs'] = data['Results'][0]['DisplayAs']
    member_info['ListAs'] = data['Results'][0]['ListAs']
    member_info['FullTitle'] = data['Results'][0]['FullTitle']

    member_info['Gender'] = data['Results'][0]['Gender']
    member_info['Party'] = data['Results'][0]['Party']
    member_info['PartyId'] = data['Results'][0]['PartyId']
    member_info['ConstituencyCountry'] = data['Results'][0]['ConstituencyCountry']

    member_info_dict[memberid] = member_info

In [5]:
uk_parliament_interactions_graph_data = {"nodes": member_info_dict, "interactions": interactions}

Saving the final dataset

In [6]:
# save as json file
with open('uk_parliament_interactions_graph_data_V4.json', 'w') as f:
    json.dump(uk_parliament_interactions_graph_data, f)